In [1]:
import pandas as pd
from tqdm import tqdm

path = "/gscratch/jamiemmt/andersonlee/image-distributionally-robust-data-join/src/generate_pairs/test_pairs.csv"
df = pd.read_csv(path)

md_path = "/gscratch/cse/lee0618/cifar-100/meta/train.csv"
md = pd.read_csv(md_path)

print(len(df))
superclass_agreement = df.loc[df["superclass_1"] == df["superclass_2"]]
label_agreement = df.loc[df["label_1"] == df["label_2"]]
print(len(superclass_agreement) / len(df)) # 0.73 (k = 4), 0.739 (k = 3), 0.744 (k = 2), 0.753 (k = 1)
print(len(label_agreement) / len(df)) # 0.62 (k = 4), 0.625 (k = 3), 0.63 (k = 2), 0.643 (k = 1)

print(df.loc[df["superclass_1"] != df["superclass_2"]].head())

79609
0.08220176110741248
0.025889032647062517
   Unnamed: 0      id_1        id_2  label_1  label_2  superclass_1  \
0           0  19_1.jpg  27_376.jpg       19       27            11   
1           1  19_1.jpg  78_262.jpg       19       78            11   
2           2  29_1.jpg  23_344.jpg       29       23            15   
3           3  29_1.jpg   8_269.jpg       29        8            15   
4           4   0_1.jpg  63_238.jpg        0       63             4   

   superclass_2  
0            15  
1            15  
2            10  
3            18  
4            12  
